# edit anywhere

Purpose: change the web however you want, save and restore it from github, treat the web like one giant web of gists.




## introduction

First attempt at this app was using megamind.bot app, the editor takes an input URL, and a filename.


 It loads a website from crawled data from Selenium/data collection.ipynb. It then sent these parameters to the client app for displaying in the same frame.

The new implementation should act more like a marketing website. Enter a URL in a box in the middle like Google, use the controls that appear on the copied page content. Use different methods to get the page content such as Selenium crawler or simple phantom browser.

After the content is crawled, make any changes that save to a single Gist for the entire domain. Load the gist from domain when the page is loaded (plugin phase 2).

TODO:

Fix SPA/PWA apps and canvas copies.

Move page processing to generalized Selenium, convert scripts.


## gist

Read and write files from gist.



### read gist file


#### the code

read gist file?


### write gist file



#### the code 



## ckeditor client

Use CKEditor and some scripts to apply some ACLs to the page and output.



### apply ACL


#### the code
apply acl to html?


### load ckeditor


#### the code


## tools


### restrain CSS

Replace all CSS rules with a container ID to restain it's affects on the page.


#### the code

restrain css? 



In [ ]:
var {JSDOM} = require('jsdom');
var _ = require('lodash');
var glob = require('glob')
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/crawls');

function noop() {}

function getAllLinks(doc) {
    var dom = new JSDOM(doc);
    var xpaths = importer.import('all elements xpath');
    xpaths.document = dom.window.document;
    xpaths.client = {
        execute: (func, ...args) => Promise.resolve({
            // TODO: Synchronous utility!?
            value: func.apply(dom.window.document, args)
        }),
        addCommand: noop
    };
    return xpaths.getAllXPath({
        links: '//a/@href|//*[@src]/@src',
        styles: '//link/@href|style/@text()'
    }); 
}

function fixLink(url, l) {
    return l.trim().match(/^\/\//ig)
       ? ('https:' + l.trim())
       : (l.trim().match(/^https?:\/\//ig)
         ? l.trim()
         : url.replace(/\/$/ig, '') + '/' + l.trim().replace(/^\//ig, ''));
}

function loadScraped(gist, file) {
    let files = {};
    const crawl = _.sortBy(
        glob.sync(
            '**/*' + gist.replace(/[^a-z0-9_-]/ig, '_') + '*',
            {cwd: project}
        ),
        a => fs.statSync(path.join(project, a)).mtime.getTime()
    ).reverse();
    // bail
    if(!crawl.length) {
        return files;
    }
    
    const entry = JSON.parse(fs.readFileSync(path.join(project, crawl[0])).toString());
    const content = entry.filter(r => r.url.includes(file)
                                 || file === 'index'
                                 && r.url.trim().replace(/https?:\/\/|\/$/ig, '').includes(gist))[0];
    if(content) {
        wrapXPath(content.content);
        const styles = content.styles.map(s => fixLink(content.url, s));
        const stylesheets = entry
            // TODO: fix this body doesn't work, remove <body tags from downloaded page
            .filter(r => styles.includes(r.url))
            .map(r => r.content.replace(/<body[^>]*?><pre[^>]*?>|<\/pre[^>]*?><\/body[^>]*?>/ig, ''))
            .filter(r => !r.match(/<[^>]*?>.*?<\/[^>]*?>/ig))
            .join('\n');
        files['index.html'] = {
            content: content.content
        };
        files['index-acl.json'] = {
            content: '[]'
        };
        files['index.css'] = {
            content: stylesheets
        };
    }
    return getAllLinks(content.content)
        .then(r => {})
        .then(() => files)
}
module.exports = loadScraped;


In [ ]:
var GitHubApi = require('github');
var ASTQ = require('astq');
var ASTY = require('asty');
var css = require('css');
var glob = require('glob');
var importer = require('../Core');
var loadScraped = importer.import('get scraped page');

// commit changes to github
function getGist(gist) {
    const github = new GitHubApi({
        host: 'api.github.com'
    });
    /*
    github.authenticate({
        type: 'basic',
        username: process.env.USERNAME,
        password: process.env.PASSWORD
    });
    */

    return github.gists.get({id: gist})
        .then(r => r.data.files);
}

function replaceSelectors(str, prefix) {
    const ast = css.parse(str);
    // TODO: add a check for media queries
    ast.stylesheet.rules.forEach(r => {
        if(typeof r.selectors === 'undefined') {
            return;
        }
        r.selectors.forEach((s, i) => {
            r.selectors[i] = prefix + ' ' + s;
        });
    })
    return css.stringify(ast);
}

// git 
function gitEditor(gist, file) {
    // TODO: use a Github repo as the input
    const HOST_ID = 'webtest123'
    gist = gist || 'act.com' || 'a572d0830ae72b962e12a57adaec7c52';
    file = file || 'index';
    return loadScraped(gist, file)
        .then(files => typeof files[ file + '-acl.json' ] !== 'undefined'
            ? Promise.resolve(files)
            : getGist(gist))
        .then(files => {
            // scan using an acl list, similar to easylist?
            // TODO: accept formats:
            //    {"selector": "selector"}
            //    {"glob-url@selector": "glob-template-path@selector"}
            //    {"selector": "html-file@selector"}
            //    {"selector": "html-file@xpath"} ?
            //    {"glob-file": {"glob-url"...} || ["selector"]}
            const paths = JSON.parse(files[ file + '-acl.json' ].content);
            const content = files[ file + '.html' ].content;
            const css = replaceSelectors(files[ file + '.css' ].content, '#' + HOST_ID);
            return {
                id: HOST_ID,
                html: '<style>' + css + '</style>' + content,
                acl: paths
            };
        })
}
module.exports = gitEditor;

if(typeof $$ !== 'undefined') {
    $$.async();
    gitEditor()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}
